In [2]:
!pip install top2vec
!pip install llvmlite==0.34

In [3]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict 
from ast import literal_eval
from collections import Counter
import re
import unicodedata
# from nlp_preprocessing import *
# from topic_modeling import *
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS, CountVectorizer
import spacy
from top2vec import Top2Vec

sp_nlp = spacy.load('en')

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_colwidth', None)

pd.reset_option('display.max_colwidth')

%load_ext autoreload
%autoreload 2

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Unbiased_news/Data/expanded_in_process.csv")
df.head()

,article,number,global_bias,title,news_source,text_ascii,text_paras,para_count,text_paras_lemma,text_paras_misc_clean,text_paras_stopwords,text_paras_no_small_words
0,0,5,From the Left,Trump Administration Drops Citizenship Questio...,HuffPost,A very sad time for America when the Supreme C...,A very sad time for America when the Supreme C...,0,a very sad time for America when the Supreme C...,a very sad time for America when the Supreme C...,sad time America Supreme Court United States a...,sad time America Supreme Court United States a...
1,0,5,From the Left,Trump Administration Drops Citizenship Questio...,HuffPost,A very sad time for America when the Supreme C...,President Donald Trump spoke out Tuesday on hi...,1,President Donald Trump speak out Tuesday on a...,President Donald Trump speak out Tuesday on a...,President Donald Trump speak Tuesday administr...,President Donald Trump speak Tuesday administr...
2,0,5,From the Left,Trump Administration Drops Citizenship Questio...,HuffPost,A very sad time for America when the Supreme C...,A very sad time for America when the Supreme C...,2,a very sad time for America when the Supreme C...,a very sad time for America when the Supreme C...,sad time America Supreme Court United States a...,sad time America Supreme Court United States a...
3,0,5,From the Left,Trump Administration Drops Citizenship Questio...,HuffPost,A very sad time for America when the Supreme C...,....to do whatever is necessary to bring this ...,3,.... to do whatever be necessary to bring this...,to do whatever be necessary to bring this mos...,whatever necessary bring vital question import...,whatever necessary bring vital question import...
4,0,5,From the Left,Trump Administration Drops Citizenship Questio...,HuffPost,A very sad time for America when the Supreme C...,"Commerce Secretary Wilbur Ross, who oversees t...",4,"Commerce Secretary Wilbur Ross , who oversee t...",Commerce Secretary Wilbur Ross who oversee th...,Commerce Secretary Wilbur Ross oversee US Cens...,Commerce Secretary Wilbur Ross oversee Census ...


In [16]:
df_no_nulls = df.dropna(subset=['text_paras_no_small_words']).reset_index()

In [17]:
df_no_nulls.sample(5)

,index,article,number,global_bias,title,news_source,text_ascii,text_paras,para_count,text_paras_lemma,text_paras_misc_clean,text_paras_stopwords,text_paras_no_small_words
218,242,9,57,From the Right,Hundreds Arrested in Moscow Protests,Fox News (Online News),Police in Moscow arrested more than 750 people...,CLICK HERE TO GET THE FOX NEWS APP,13,click here to get the FOX news APP,click here to get the FOX news APP,get FOX news APP,get FOX news APP
24772,26686,1038,3374,From the Left,House Committee to Vote on Holding Barr in Con...,New York Times (News),WASHINGTON The Trump administration ruled out...,"The Judiciary Committees chairman, Representat...",13,"the Judiciary Committees chairman , Representa...",the Judiciary Committees chairman Representat...,Judiciary Committees chairman Representative J...,Judiciary Committees chairman Representative J...
5345,5926,202,856,From the Left,Supreme Court Rules Trump's Tax Returns Can Be...,Washington Post,"In the other, the court said the restrictions ...",The court said Congress cannot seek the presid...,39,the court say Congress can not seek the presid...,the court say Congress can not seek the presid...,court Congress seek president information part...,court Congress seek president information part...
22291,23868,935,3128,From the Right,Healthcare Debate Moves to Main Spotlight,Washington Times,WASHINGTON A federal judges ruling that the O...,"Douglas Holtz-Eakin, top policy adviser to Rep...",10,"Douglas Holtz - Eakin , top policy adviser to ...",Douglas Holtz Eakin top policy adviser to Re...,Douglas Holtz Eakin top policy adviser Republi...,Douglas Holtz Eakin top policy adviser Republi...
10867,11723,450,1704,From the Left,DOJ: Ferguson Police Racially Biased,New York Times (News),"WASHINGTON Ferguson, Mo., is a third white, b...",The report calls for city officials to acknowl...,13,the report call for city official to acknowled...,the report call for city official to acknowled...,report call city official acknowledge police d...,report call city official acknowledge police d...


In [18]:
doc = df_no_nulls.text_paras.tolist()

In [19]:
model = Top2Vec(documents=doc, speed="learn", workers=8)

2020-11-29 21:26:16,647 - top2vec - INFO - Pre-processing documents for training
2020-11-29 21:26:19,018 - top2vec - INFO - Creating joint document/word embedding
2020-11-29 21:28:43,947 - top2vec - INFO - Creating lower dimension embedding of documents
2020-11-29 21:29:01,160 - top2vec - INFO - Finding dense areas of documents
2020-11-29 21:29:04,073 - top2vec - INFO - Finding topics


In [10]:
model.get_num_topics()

243

In [11]:
topic_words, word_scores, topic_nums = model.get_topics()

In [23]:
df_no_nulls.shape

(25863, 13)

In [30]:
import pickle

with open("/content/drive/MyDrive/Unbiased_news/Data/top2vec_model.pickle", 'wb') as model_file:
    pickle.dump(model, model_file)

In [25]:
docs, doc_topic_score, doc_ids = model.search_documents_by_topic(topic_num=5, num_docs=330, return_documents=True, reduced=False)

In [27]:
help(model)

Help on Top2Vec in module top2vec.Top2Vec object:

class Top2Vec(builtins.object)
 |  Top2Vec
 |  
 |  Creates jointly embedded topic, document and word vectors.
 |  
 |  
 |  Parameters
 |  ----------
 |  embedding_model: string
 |      This will determine which model is used to generate the document and
 |      word embeddings. The valid string options are:
 |  
 |          * doc2vec
 |          * universal-sentence-encoder
 |          * universal-sentence-encoder-multilingual
 |          * distiluse-base-multilingual-cased
 |  
 |      For large data sets and data sets with very unique vocabulary doc2vec
 |      could produce better results. This will train a doc2vec model from
 |      scratch. This method is language agnostic. However multiple languages
 |      will not be aligned.
 |  
 |      Using the universal sentence encoder options will be much faster since
 |      those are pre-trained and efficient models. The universal sentence
 |      encoder options are suggested for sm

In [28]:
model.get_topic_sizes()

(array([755, 457, 454, 446, 348, 330, 315, 298, 294, 292, 287, 280, 273,
        267, 249, 238, 233, 229, 217, 212, 202, 200, 192, 191, 185, 184,
        176, 174, 173, 167, 164, 162, 162, 158, 156, 156, 153, 146, 140,
        140, 139, 138, 137, 136, 135, 133, 132, 132, 130, 129, 127, 127,
        126, 126, 123, 123, 120, 118, 117, 116, 116, 116, 115, 115, 115,
        113, 113, 113, 111, 111, 109, 109, 108, 108, 107, 107, 106, 105,
        105, 104, 103, 101, 100,  99,  96,  96,  96,  95,  94,  94,  94,
         93,  93,  91,  91,  90,  90,  90,  90,  90,  89,  89,  88,  88,
         87,  87,  87,  87,  86,  86,  86,  86,  85,  84,  84,  84,  84,
         84,  84,  83,  83,  83,  82,  81,  81,  81,  81,  80,  79,  79,
         79,  79,  79,  78,  78,  78,  78,  77,  77,  77,  77,  76,  75,
         75,  75,  75,  75,  74,  74,  74,  74,  72,  71,  71,  71,  71,
         70,  70,  69,  69,  69,  69,  68,  68,  68,  67,  67,  66,  66,
         66,  66,  66,  65,  64,  64,  64,  64,  64

In [12]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=48, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 18894, Score: 0.7890528440475464
-----------
She said Mr. Trump should be providing more money for the police, emergency medical workers and families in need of treatment.
-----------

Document: 22643, Score: 0.7260197997093201
-----------
Employees at those agencies deemed essential will continue working without pay, including many Transportation Security Administration workers dealing with the influx of holiday travelers. After every previous shutdown, Congress has passed legislation to retroactively pay employees.
-----------

Document: 9040, Score: 0.7209782004356384
-----------
Mr. Carney argued Wednesday that the law is providing consumers with more choices, not a disincentive to work.
-----------

Document: 22552, Score: 0.7207549810409546
-----------
Employees at those agencies deemed essential will continue working without pay, including many Transportation Security Administration workers dealing with the influx of holiday travelers. After every previous shutdown, Co